# Lab3

Please run the code cells sequentially.


In [1]:
# Install the required packages
! pip install -r requirements.txt

In [2]:
# Import necessary libraries
from utils.db_data_loader import DBDataLoader
import tabulate as tb
from utils.employee_model import EmployeeDataProcessor



## Database Creation
The `employees` table is created in a PostgreSQL database using the following schema:
- `employee_id`: Integer, primary key, auto-incremented
- `name`: String
- `position`: String (IT-related job titles)
- `start_date`: Date (between 2015 and 2024)
- `salary`: Integer ($60,000–$200,000)
- `department_id`: Integer

The `departments` table is created in a PostgreSQL database using the following schema:
- `department_id`: Integer, primary key, auto-incremented
- `department_name`: String
- `location`: String
- `budget`: Integer


## Database Connection
The connection string for the Neon.tech PostgreSQL database is stored in `utils/constants.py` as `CONNECTION_STRING`.

## Fake Data Generation
Fake employee data is generated using the `Faker` library in the `EmployeeSeeder` class (`utils/fake_data_creator.py`). It creates 100 records with realistic names, IT job titles, start dates, and salaries for the employee table and 10 rows for the department table.

## Data Insertion
To insert the generated data into the database, call the `generate_and_insert()` static method of the `EmployeeSeeder` class. This method connects to the database and populates the `employees` table with the fake data and populates department table with the corresponding departments of employees.

In [3]:
#Uncomment the 2 lines below to run the data insertion (100 records)
# from utils.fake_data_creator import FakeDataSeeder
# FakeDataSeeder.generate_and_insert()

## Connect and Load Data

`DBDataLoader`, a class to connect to the PostgreSQL database and load data from the employees and departments tables into Pandas DataFrames.

Methods
`__init__(self)`
Initializes the database connection using the connection string from utils/constants.py.

`load_employees(self)`
Loads all records from the employees table into a Pandas DataFrame, closes the connection, and returns the DataFrame.

`load_departments(self)`
Loads all records from the departments table into a Pandas DataFrame, closes the connection, and returns the DataFrame.

`close(self)`
Closes the database connection.

In [4]:
# Load employees and departments data into DataFrames
employees_df = DBDataLoader().load_employees()
departments_df = DBDataLoader().load_departments()

#  Sorting employees by salary to show dirty records
sorted_employee_df = employees_df.sort_values(by='salary', na_position='first')

print(tb.tabulate(sorted_employee_df.head(),headers="keys",tablefmt="outline"))



+----+---------------+-----------------+--------------------+--------------+----------+-----------------+
|    |   employee_id | name            | position           | start_date   |   salary |   department_id |
+====+===============+=================+====================+==============+==========+=================+
| 98 |           402 | Debra Davila    | Frontend Developer | 2023-02-18   |      nan |              74 |
| 99 |           404 | Stephanie Giles | Software Engineer  | 2024-04-24   |      nan |              80 |
|  1 |           403 | Alexandra Blake | QA Engineer        | 2016-04-17   |    60043 |              80 |
| 51 |           454 | Hannah Ramirez  | Frontend Developer | 2021-05-17   |    61511 |              76 |
| 30 |           433 | Jeffrey Johnson | Network Engineer   | 2020-02-12   |    61584 |              79 |
+----+---------------+-----------------+--------------------+--------------+----------+-----------------+


## Exploratory Data Analysis (EDA)

1. Data Collection - The data was sourced from a PostgreSQL database hosted on Neon.tech. The employees and departments tables were populated with synthetic data using the Faker library via the FakeDataSeeder class in utils/fake_data_creator.py. Data was loaded into Pandas DataFrames using the DBDataLoader class.
2. Data Cleaning - Duplicate or missing records removed, filled missing sallary fields with median. This is done using `remove_dupes_fill_salary` function in `EmployeeDataProcessor` class.
3. Data Transformation - New columns were created for analysis. The start_date column was converted to a datetime type. Job titles were standardized to ensure consistency. These steps are done in `clean_and_process` method of `EmployeeDataProcessor` class.
4. Feature Engineering - Additional features were derived to enhance analysis. For example, a years_of_service column was calculated by subtracting the start_date from the current date, providing insight into employee experience.
5. Scaling - salary was scaled using normalization techniques (Min-Max scaling) to ensure comparability and to prepare the data for machine learning algorithms if needed. This is done in `scale_salary` method of `EmployeeDataProcessor` class.





In the cleaned dataframe, you can see that 'years_of_service' is calculated, and the 'salary' column is normalized and added as a new column. Also the missing salary values are also now filled with median salary value.

In [6]:
employee_data_processor = EmployeeDataProcessor(sorted_employee_df)
employee_processed_df = employee_data_processor.clean_and_process()
print(tb.tabulate(employee_processed_df.head(),headers="keys",tablefmt="outline"))


+----+---------------+-----------------+--------------------+---------------------+----------+-----------------+--------------------+-----------------+
|    |   employee_id | name            | position           | start_date          |   salary |   department_id |   years_of_service |   salary_scaled |
+====+===============+=================+====================+=====================+==========+=================+====================+=================+
| 98 |           402 | Debra Davila    | Frontend Developer | 2023-02-18 00:00:00 |   123692 |              74 |                  2 |       0.45715   |
| 99 |           404 | Stephanie Giles | Software Engineer  | 2024-04-24 00:00:00 |   123692 |              80 |                  1 |       0.45715   |
|  1 |           403 | Alexandra Blake | Qa Engineer        | 2016-04-17 00:00:00 |    60043 |              80 |                  9 |       0         |
| 51 |           454 | Hannah Ramirez  | Frontend Developer | 2021-05-17 00:00:00 |    6